In [2]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [3]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64 (or choose another size)
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize pixel values
])

In [4]:
train_data = datasets.ImageFolder(root='train', transform=transform)

In [5]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class SimpsonsCNN(nn.Module):
    def __init__(self):
        super(SimpsonsCNN, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)

        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 8 * 8, 512)  # Adjust the size
        self.fc2 = nn.Linear(512, 42)  # num_classes = number of characters

    def forward(self, x):
        # Apply convolutions and pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        # Flatten the output for the dense layers
        x = x.view(-1, 128 * 8 * 8)  # Adjust the flattening size

        # Apply fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [7]:
import torch.optim as optim

model = SimpsonsCNN()
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    for images, labels in train_loader:
        optimizer.zero_grad()  # Zero the parameter gradients

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Save the trained model
torch.save(model.state_dict(), 'simpsons_model.pth')

Epoch [1/5], Loss: 0.0000
Epoch [2/5], Loss: 0.0000


KeyboardInterrupt: 